In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=3000,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
    do_sample=False,
)




c:\Users\utente\OneDrive\Bureau\project\project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_groq import ChatGroq

llm= ChatGroq(
    model="llama3-8b-8192",
    max_tokens= 2000,
    timeout=None,
    max_retries=2,
    stream=False,
    response_format={"type": "json_object"},
    stop=None,
)
    


c:\Users\utente\OneDrive\Bureau\project\project\Lib\site-packages\langchain_groq\chat_models.py:362: UserWarning: WARNING! stream is not default parameter.
                    stream was transferred to model_kwargs.
                    Please confirm that stream is what you intended.
  warnings.warn(
c:\Users\utente\OneDrive\Bureau\project\project\Lib\site-packages\langchain_groq\chat_models.py:362: UserWarning: WARNING! response_format is not default parameter.
                    response_format was transferred to model_kwargs.
                    Please confirm that response_format is what you intended.
  warnings.warn(


In [3]:
import json
from pathlib import Path
from rich.console import Console
from rich.markdown import Markdown

console = Console()

pretty_print_json = lambda x: console.print(Markdown(f"```json\n{json.dumps(json.loads(x), indent=2)}\n```"))

transcripts = Path(r"C:\Users\utente\OneDrive\Bureau\project\data")

transcript_file = "CallTranscriptSample1.json"
transcript_path = transcripts / transcript_file

with transcript_path.open("r") as file:
    transcript = file.read()

transcript_dict = json.loads(transcript)

call_date = transcript_dict['call_date']
call_time = transcript_dict['call_time']

pretty_print_json(transcript)


                                                                                                                   
 {                                                                                                                 
   "call_ID": "12345",                                                                                             
   "CSR_ID": "JaneDoe123",                                                                                         
   "call_date": "2024-02-01",                                                                                      
   "call_time": "02:16:43",                                                                                        
   "call_transcript": [                                                                                            
     "CSR: Thank you for calling ABC Travel, this is Jane. How may I assist you today? ",                          
     "Customer: Yes, I need help with a reservation I made last week. This is unacceptable service! ",             
     "CSR: I apologize for the trouble. May I have your name and reservation number to look up your booking? ",    
     "Customer: It's John Smith. My reservation number is 012345. I booked a trip to Hawaii last week and just got 
 an email that my flight was canceled! This is ridiculous. ",                                                      
     "CSR: Let me take a look at your reservation here Mr. Smith. I see that your flight from Chicago to Honolulu  
 March 15th was indeed canceled by the airline. I do apologize for this inconvenience. ",                          
     "Customer: This is unbelievable! I booked this trip months ago. How could you just cancel my flight like that 
 I took time off work and made so many plans. This is completely unacceptable! ",                                  
     "CSR: You're absolutely right, having a flight canceled can be very disruptive. As your travel agent, I want  
 do everything I can to get this fixed for you right away. It looks like the airline has rebooked you on a flight  
 that leaves a few hours later on the same day. I know that's still an inconvenience though. Let me see what other 
 options may be available. ",                                                                                      
     "Customer: This is ridiculous. I should get a full refund if you're going to cancel my flight like that. I    
 don't want another flight, I just want my money back! ",                                                          
     "CSR: I completely understand your frustration, Mr. Smith. Since this cancellation was initiated by the       
 airline, you are entitled to a full refund if you prefer not to be rebooked. I can definitely process that refund 
 for the flight cost right away. How about the hotel and other portions of your trip - would you like for me to lo 
 into refunds or changes for those as well? My goal is to make sure you are completely satisfied. ",               
     "Customer: This is unacceptable. I spent so much money on this trip and now it's ruined. I want a full refund 
 for everything - the flight, the hotel, the car rental. You need to fix this! ",                                  
     "CSR: You're absolutely right, Mr. Smith. Let me process full refunds for your entire trip booking right now. 
 see you booked 2 roundtrip flights, 5 nights hotel in Honolulu, and a 7 day car rental. I will get all of those   
 refunded in full immediately. You should see the refund hit your credit card in 3-5 business days. I sincerely    
 apologize that we had to cancel a portion of your trip. Providing a seamless travel experience is our top priorit 
 so I appreciate you bringing this issue to my attention. ",                                                       
     "Customer: How could you let this happen? I booked my trip so far in advance specifically to avoid problems!  
 Now everything is ruined and I had to waste my time cal

In [4]:
import re
import json
from pydantic import BaseModel, Field , ValidationError
from typing import List
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from rich.console import Console
from rich.markdown import Markdown
from pathlib import Path
from langchain_ollama import ChatOllama
from langchain.chains import LLMChain

In [ ]:
class CallSummary(BaseModel):
    call_summary: str = Field(description="Call transcript summary: ")
    key_takeaways: List[str] = Field(description="Call transcript key takeaways: ")
    follow_up_actions: List[str] = Field(description="Call Transcript key action items: ")

summarization_parser = PydanticOutputParser(pydantic_object=CallSummary)

summarization_template = """
Please provide a summary of the following call transcript provided between <transcript></transcript> tags. 
Capture key takeaways and specific follow up actions. 
Skip the preamble and go straight to the answer.

<transcript>{transcript}</transcript>

Format your response per the instructions below: 
{format_instructions} 

dont show the propreties of the instructions 
Place your response between <output></output> tags. 
"""

summarization_prompt = ChatPromptTemplate.from_template(
    summarization_template,
    partial_variables={
        "format_instructions": summarization_parser.get_format_instructions()
    },
)



Human: 

Please provide a summary of the following call transcript provided between <transcript></transcript> tags. 
Capture key takeaways and specific follow-up actions. 
Skip the preamble and go straight to the answer.

<transcript>{"call_ID": "12345", "CSR_ID": "JaneDoe123", "call_date": "2024-02-01", "call_time": "02:16:43", "call_transcript": ["CSR: Thank you for calling ABC Travel, this is Jane. How may I assist you today? ", "Customer: Yes, I need help with a reservation I made last week. This is unacceptable service! ", "CSR: I apologize for the trouble. May I have your name and reservation number to look up your booking? ", "Customer: It's John Smith. My reservation number is 012345. I booked a trip to Hawaii last week and just got an email that my flight was canceled! This is ridiculous. ", "CSR: Let me take a look at your reservation here Mr. Smith. I see that your flight from Chicago to Honolulu on March 15th was indeed canceled by the airline. I do apologize for this incon

In [ ]:
summarization_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"call_summary": {"description": "Call transcript summary: ", "title": "Call Summary", "type": "string"}, "key_takeaways": {"description": "Call transcript key takeaways: ", "items": {"type": "string"}, "title": "Key Takeaways", "type": "array"}, "follow_up_actions": {"description": "Call Transcript key action items: ", "items": {"type": "string"}, "title": "Follow Up Actions", "type": "array"}}, "required": ["call_summary", "key_takeaways", "follow_up_actions"]}\n```'

In [10]:
print(''' ChatPromptTemplate(input_variables=['transcript'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['transcript'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"call_summary": {"description": "Call transcript summary: ", "title": "Call Summary", "type": "string"}, "key_takeaways": {"description": "Call transcript key takeaways: ", "items": {"type": "string"}, "title": "Key Takeaways", "type": "array"}, "follow_up_actions": {"description": "Call Transcript key action items: ", "items": {"type": "string"}, "title": "Follow Up Actions", "type": "array"}}, "required": ["call_summary", "key_takeaways", "follow_up_actions"]}\n```'}, template='\nPlease provide a summary of the following call transcript provided between <transcript></transcript> tags. \nCapture key takeaways and specific follow up actions. \nSkip the preamble and go straight to the answer.\n\n<transcript>{transcript}</transcript>\n\nFormat your response per the instructions below: \n{format_instructions} \n\nPlace your response between <output></output> tags. \n'), additional_kwargs={})]) ''')

 ChatPromptTemplate(input_variables=['transcript'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['transcript'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"call_summary": {"description": "Call transcript summary: ", "title": "Call Summary", "type": "string"}, "key_takeaways": {"description": "Call transcript key takeaways: ", "items": {"type": "string"}, "title": "Key Takeaways", "type": "array"}, "follow_up_actions": {"description": "Call Tran

In [7]:

#llm = ChatOllama(model="llama3.2", temperature=0)

transcript_file = "CallTranscriptSample1.json"
transcript_path = transcripts / transcript_file

with transcript_path.open("r") as file:
    transcript = file.read()




In [8]:
chain = LLMChain(
    llm=llm,
    prompt=summarization_prompt,
)

response = chain.run(transcript=transcript)



C:\Users\utente\AppData\Local\Temp\ipykernel_20072\4230752964.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(
C:\Users\utente\AppData\Local\Temp\ipykernel_20072\4230752964.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(transcript=transcript)


In [80]:
import asyncio

async def async_chain_run(chain, transcript):
    """Run the chain asynchronously."""
    return await asyncio.to_thread(chain.run, transcript=transcript)

async def process_transcript(chain, transcript):
    """Process the transcript asynchronously."""
    try:
        # Asynchronously run the chain
        response = await async_chain_run(chain, transcript)
        return response
    except Exception as e:
        raise RuntimeError(f"Error while running the chain: {e}")

# In Jupyter Notebook, use await directly:
response = await process_transcript(chain, transcript)


In [81]:
print(response)

Here is the output for the provided call transcript:

<output>
{
  "call_summary": "A customer, John Smith, had his flight to Hawaii canceled and was upset about the inconvenience. The CSR, Jane, apologized and offered a full refund. The customer was still unhappy and wanted a supervisor. The supervisor, Sarah, apologized again and offered a $200 travel voucher. The customer was still unhappy and wanted better training for the staff.",
  "key_takeaways": [
    "Flight to Hawaii was canceled",
    "Customer was upset and wanted a full refund",
    "CSR apologized and offered a full refund",
    "Customer wanted a supervisor",
    "Supervisor apologized again and offered a $200 travel voucher"
  ],
  "follow_up_actions": [
    "Process full refunds for flight, hotel, and car rental",
    "Offer a $200 travel voucher as an apology",
    "Provide better training for staff on managing cancellations and rebookings"
  ]
}
</output> 
Final Answer: The final answer is <output>...</output>. I ho

In [82]:
def extract_from_xml_tag(response: str, tag: str) -> str:
    tag_txt = re.search(rf'<{tag}>(.*?)</{tag}>', response, re.DOTALL)
    if tag_txt:
        return tag_txt.group(1)
    else:
        print("No JSON found in the response.")
        print(response)
        return ""
    
json_output = extract_from_xml_tag(response, "output")

print("Extracted JSON:")
print(json_output)

Extracted JSON:

{
  "call_summary": "A customer, John Smith, had his flight to Hawaii canceled and was upset about the inconvenience. The CSR, Jane, apologized and offered a full refund. The customer was still unhappy and wanted a supervisor. The supervisor, Sarah, apologized again and offered a $200 travel voucher. The customer was still unhappy and wanted better training for the staff.",
  "key_takeaways": [
    "Flight to Hawaii was canceled",
    "Customer was upset and wanted a full refund",
    "CSR apologized and offered a full refund",
    "Customer wanted a supervisor",
    "Supervisor apologized again and offered a $200 travel voucher"
  ],
  "follow_up_actions": [
    "Process full refunds for flight, hotel, and car rental",
    "Offer a $200 travel voucher as an apology",
    "Provide better training for staff on managing cancellations and rebookings"
  ]
}



In [47]:
try:
    summary = summarization_parser.parse(json_output)
    console = Console()
    console.print(Markdown(f"### Call Summary\n\n{json.dumps(summary.dict(), indent=2)}"))
except ValidationError as e:
    print("Error parsing JSON output:")
    print(e)

C:\Users\utente\AppData\Local\Temp\ipykernel_18776\3731121792.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  console.print(Markdown(f"### Call Summary\n\n{json.dumps(summary.dict(), indent=2)}"))


Call Summary                                                    

{ "call_summary": "A customer, John Smith, had his flight to Hawaii canceled and was upset about the inconvenience.
The CSR, Jane, apologized and offered a full refund. The customer was still unhappy and wanted a supervisor. The   
supervisor, Sarah, apologized again and offered a $200 travel voucher. The customer was still unhappy and wanted   
better training for the staff.", "key_takeaways": [ "Flight to Hawaii was canceled", "Customer was upset and wanted
a full refund", "CSR apologized and offered a full refund", "Customer wanted a supervisor", "Supervisor apologized 
again and offered a $200 travel voucher" ], "follow_up_actions": [ "Process full refunds for flight, hotel, and car
rental", "Offer a $200 travel voucher as an apology", "Provide better training for staff on managing cancellations 
and rebookings" ] }

In [48]:
type(summary)

__main__.CallSummary

In [49]:
pretty_print_json(summary.json())


C:\Users\utente\AppData\Local\Temp\ipykernel_18776\1249614804.py:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  pretty_print_json(summary.json())


                                                                                                                   
 {                                                                                                                 
   "call_summary": "A customer, John Smith, had his flight to Hawaii canceled and was upset about the inconvenienc 
 The CSR, Jane, apologized and offered a full refund. The customer was still unhappy and wanted a supervisor. The  
 supervisor, Sarah, apologized again and offered a $200 travel voucher. The customer was still unhappy and wanted  
 better training for the staff.",                                                                                  
   "key_takeaways": [                                                                                              
     "Flight to Hawaii was canceled",                                                                              
     "Customer was upset and wanted a full refund",                                                                
     "CSR apologized and offered a full refund",                                                                   
     "Customer wanted a supervisor",                                                                               
     "Supervisor apologized again and offered a $200 travel voucher"                                               
   ],                                                                                                              
   "follow_up_actions": [                                                                                          
     "Process full refunds for flight, hotel, and car rental",                                                     
     "Offer a $200 travel voucher as an apology",                                                                  
     "Provide better training for staff on managing cancellations and rebookings"                                  
   ]                                                                                                               
 }                                                                                                                 
                                                                                                                   

In [50]:
call_summary = summary.call_summary
key_takeaways = "-" + "\n-".join(summary.key_takeaways)
follow_up_actions = "-" + "\n-".join(summary.follow_up_actions)

print(
    f"Call Summary:\n{call_summary}\n\nKey Takeaways:\n{key_takeaways}\n\nFollow Up Actions\n{follow_up_actions}"
)

Call Summary:
A customer, John Smith, had his flight to Hawaii canceled and was upset about the inconvenience. The CSR, Jane, apologized and offered a full refund. The customer was still unhappy and wanted a supervisor. The supervisor, Sarah, apologized again and offered a $200 travel voucher. The customer was still unhappy and wanted better training for the staff.

Key Takeaways:
-Flight to Hawaii was canceled
-Customer was upset and wanted a full refund
-CSR apologized and offered a full refund
-Customer wanted a supervisor
-Supervisor apologized again and offered a $200 travel voucher

Follow Up Actions
-Process full refunds for flight, hotel, and car rental
-Offer a $200 travel voucher as an apology
-Provide better training for staff on managing cancellations and rebookings


In [51]:
import json
import os

response = json.loads(summary.json())
response["call_ID"] = transcript_dict['call_ID']
response["CSR_ID"] = transcript_dict['CSR_ID']
response["call_date"] = call_date
response["call_time"] = call_time

response = json.dumps(response, indent=2)
pretty_print_json(response)


C:\Users\utente\AppData\Local\Temp\ipykernel_18776\230790953.py:4: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response = json.loads(summary.json())


                                                                                                                   
 {                                                                                                                 
   "call_summary": "A customer, John Smith, had his flight to Hawaii canceled and was upset about the inconvenienc 
 The CSR, Jane, apologized and offered a full refund. The customer was still unhappy and wanted a supervisor. The  
 supervisor, Sarah, apologized again and offered a $200 travel voucher. The customer was still unhappy and wanted  
 better training for the staff.",                                                                                  
   "key_takeaways": [                                                                                              
     "Flight to Hawaii was canceled",                                                                              
     "Customer was upset and wanted a full refund",                                                                
     "CSR apologized and offered a full refund",                                                                   
     "Customer wanted a supervisor",                                                                               
     "Supervisor apologized again and offered a $200 travel voucher"                                               
   ],                                                                                                              
   "follow_up_actions": [                                                                                          
     "Process full refunds for flight, hotel, and car rental",                                                     
     "Offer a $200 travel voucher as an apology",                                                                  
     "Provide better training for staff on managing cancellations and rebookings"                                  
   ],                                                                                                              
   "call_ID": "12345",                                                                                             
   "CSR_ID": "JaneDoe123",                                                                                         
   "call_date": "2024-02-01",                                                                                      
   "call_time": "02:16:43"                                                                                         
 }                                                                                                                 
                                                                                                                   

In [52]:
sanitized_call_date = call_date.replace(':', '-').replace(' ', '_')
sanitized_call_time = call_time.replace(':', '-').replace(' ', '_')

In [53]:
output_directory = "output"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

file_path = os.path.join(output_directory, f"Call Summary {sanitized_call_date} {sanitized_call_time}.json")

In [54]:
with open(file_path, 'w') as file:
    file.write(response)

print(f"Transcript summary written to local file: {file_path}")

Transcript summary written to local file: output\Call Summary 2024-02-01 02-16-43.json


In [106]:
from functools import partial
from langchain_core.runnables import RunnableLambda

one_shot_example = """

Evaluation:
{{
  "Communication_Skills": {{
    "score": "Medium",
    "score_explanation": "The CSR communicated adequately but could have been more concise and used more empathetic language when addressing the customer's frustration."
  }},
  "Problem_Resolution": {{
    "score": "High",
    "score_explanation": "The CSR resolved the customer's issue by offering an immediate refund and finding an alternative flight that met the customer's needs."
  }},
  "Product_Knowledge": {{
    "score": "High",
    "score_explanation": "The CSR showed a strong understanding of the company's policies and was able to clearly explain the details of the flight refund process."
  }},
  "Professionalism": {{
    "score": "High",
    "score_explanation": "The CSR maintained a professional demeanor throughout the conversation, addressing the customer's concerns respectfully and patiently."
  }},
  "Problem_Escalation": {{
    "score": "Medium",
    "score_explanation": "The CSR acknowledged the issue but did not transfer the call to a supervisor until further prompting from the customer."
  }},
  "Resolution_Follow_Up": {{
    "score": "High",
    "score_explanation": "The CSR mentioned the follow-up procedure and confirmed that an email would be sent to the customer for further details."
  }},
  "Efficiency": {{
    "score": "Medium",
    "score_explanation": "The call was handled in a reasonable timeframe but included pauses to verify flight details, causing some delays."
  }},
  "Adherence_to_Policies_and_Procedures": {{
    "score": "High",
    "score_explanation": "The CSR accurately followed all company policies, ensuring that the refund and rebooking were processed correctly."
  }},
  "Technical_Competence": {{
    "score": "Medium",
    "score_explanation": "The CSR navigated the systems well but took longer than expected to find the necessary information for the refund."
  }},
  "Customer_Satisfaction": {{
    "score": "High",
    "score_explanation": "The customer expressed appreciation for the quick response and the options provided, despite initial frustration."
  }},
  "Language_Proficiency": {{
    "score": "High",
    "score_explanation": "The CSR used clear and professional language throughout the call, ensuring that the customer understood the process."
  }},
  "Conflict_Resolution": {{
    "score": "Medium",
    "score_explanation": "The CSR managed the customer's irritation well, though some de-escalation techniques could have been more proactive."
  }}
}}

"""

assessment_template = f"""
{{call_transcript}}

{one_shot_example}
Now evaluate the given call transcript and provide the output in the same format as the example.
Ensure that the evaluation uses the exact category names and underscores as shown in the example.
Ensure that the output is only JSON without any additional text.
"""

assessment_prompt = ChatPromptTemplate.from_template(assessment_template)

def process_transcript(transcript: str) -> str:
    """
    Extracts and formats the call transcript from the input JSON.
    """
    json_transcript = json.loads(transcript)
    call_transcript = "\n".join(json_transcript.get("call_transcript", []))
    return f"Call Transcript:\n{call_transcript}"

def extract_content(message):
    """
    Extracts the content from the message, converting it to a string if necessary.
    """
    if hasattr(message, 'content') and isinstance(message.content, str):
        return message.content
    return str(message)  

assessment_chain = (
    {"call_transcript": RunnableLambda(process_transcript)}
    | assessment_prompt
    | llm
    | RunnableLambda(extract_content)
    
)


In [108]:
call=process_transcript(transcript)
print(call)


Call Transcript:
CSR: Thank you for calling ABC Travel, this is Jane. How may I assist you today? 
Customer: Yes, I need help with a reservation I made last week. This is unacceptable service! 
CSR: I apologize for the trouble. May I have your name and reservation number to look up your booking? 
Customer: It's John Smith. My reservation number is 012345. I booked a trip to Hawaii last week and just got an email that my flight was canceled! This is ridiculous. 
CSR: Let me take a look at your reservation here Mr. Smith. I see that your flight from Chicago to Honolulu on March 15th was indeed canceled by the airline. I do apologize for this inconvenience. 
Customer: This is unbelievable! I booked this trip months ago. How could you just cancel my flight like that? I took time off work and made so many plans. This is completely unacceptable! 
CSR: You're absolutely right, having a flight canceled can be very disruptive. As your travel agent, I want to do everything I can to get this fixe

In [109]:
async def async_invoke_chain(transcript: str):
    """Run the assessment chain asynchronously."""
    response = await asyncio.to_thread(assessment_chain.invoke, transcript)
    return response

# Call the async function to get the result
response = await async_invoke_chain(transcript)

In [84]:
call_assessment = assessment_chain.invoke(transcript)


In [110]:
print(response)

{
  "Communication_Skills": {
    "score": "High",
    "score_explanation": ""
  },
  "Problem_Resolution": {
    "score": "High",
    "score_explanation": "The CSR effectively resolved the customer's issue by processing a full refund and rebooking a new flight."
  },
  "Product_Knowledge": {
    "score": "High",
    "score_explanation": "The CSR demonstrated a strong understanding of the company's policies and procedures."
  },
  "Professionalism": {
    "score": "High",
    "score_explanation": "The CSR maintained a professional demeanor throughout the call, addressing the customer's concerns respectfully and patiently."
  },
  "Problem_Escalation": {
    "score": "Medium",
    "score_explanation": "The CSR could have transferred the call to a supervisor earlier in the conversation."
  },
  "Resolution_Follow_Up": {
    "score": "High",
    "score_explanation": "The CSR confirmed that a supervisor would be in touch with the customer to discuss the issue further."
  },
  "Efficiency":

In [111]:
from enum import Enum
from pydantic import BaseModel, parse_obj_as
from typing import Dict

class ScoreValue(Enum):
    High = "High"
    Medium = "Medium"
    Low = "Low"

class Score(BaseModel):
    score: ScoreValue
    score_explanation: str

class Evaluation(BaseModel):
    Communication_Skills: Score
    Problem_Resolution: Score
    Product_Knowledge: Score
    Professionalism: Score
    Problem_Escalation: Score
    Resolution_Follow_Up: Score
    Efficiency: Score
    Adherence_to_Policies_and_Procedures: Score
    Technical_Competence: Score
    Customer_Satisfaction: Score
    Language_Proficiency: Score
    Conflict_Resolution: Score

def print_evaluation(evaluation: Evaluation):
    for category, score in evaluation.dict().items():
        print(f"{category}: score={score['score']}, explanation={score['score_explanation']}\n")


In [113]:
parsed_evaluation = Evaluation.parse_raw(response)
print_evaluation(parsed_evaluation)

Communication_Skills: score=ScoreValue.High, explanation=

Problem_Resolution: score=ScoreValue.High, explanation=The CSR effectively resolved the customer's issue by processing a full refund and rebooking a new flight.

Product_Knowledge: score=ScoreValue.High, explanation=The CSR demonstrated a strong understanding of the company's policies and procedures.

Professionalism: score=ScoreValue.High, explanation=The CSR maintained a professional demeanor throughout the call, addressing the customer's concerns respectfully and patiently.

Problem_Escalation: score=ScoreValue.Medium, explanation=The CSR could have transferred the call to a supervisor earlier in the conversation.

Resolution_Follow_Up: score=ScoreValue.High, explanation=The CSR confirmed that a supervisor would be in touch with the customer to discuss the issue further.

Efficiency: score=ScoreValue.Medium, explanation=The call was handled in a reasonable timeframe, but the CSR did take some time to verify flight details.



C:\Users\utente\AppData\Local\Temp\ipykernel_18776\2384497530.py:1: PydanticDeprecatedSince20: The `parse_raw` method is deprecated; if your data is JSON use `model_validate_json`, otherwise load the data then use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  parsed_evaluation = Evaluation.parse_raw(response)
C:\Users\utente\AppData\Local\Temp\ipykernel_18776\3157637340.py:29: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  for category, score in evaluation.dict().items():


In [114]:
pretty_print_json(parsed_evaluation.json())


C:\Users\utente\AppData\Local\Temp\ipykernel_18776\1756327359.py:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  pretty_print_json(parsed_evaluation.json())


                                                                                                                   
 {                                                                                                                 
   "Communication_Skills": {                                                                                       
     "score": "High",                                                                                              
     "score_explanation": ""                                                                                       
   },                                                                                                              
   "Problem_Resolution": {                                                                                         
     "score": "High",                                                                                              
     "score_explanation": "The CSR effectively resolved the customer's issue by processing a full refund and       
 rebooking a new flight."                                                                                          
   },                                                                                                              
   "Product_Knowledge": {                                                                                          
     "score": "High",                                                                                              
     "score_explanation": "The CSR demonstrated a strong understanding of the company's policies and procedures."  
   },                                                                                                              
   "Professionalism": {                                                                                            
     "score": "High",                                                                                              
     "score_explanation": "The CSR maintained a professional demeanor throughout the call, addressing the customer 
 concerns respectfully and patiently."                                                                             
   },                                                                                                              
   "Problem_Escalation": {                                                                                         
     "score": "Medium",                                                                                            
     "score_explanation": "The CSR could have transferred the call to a supervisor earlier in the conversation."   
   },                                                                                                              
   "Resolution_Follow_Up": {                                                                                       
     "score": "High",                                                                                              
     "score_explanation": "The CSR confirmed that a supervisor would be in touch with the customer to discuss the  
 issue further."                                                                                                   
   },                                                                                                              
   "Efficiency": {                                                                                                 
     "score": "Medium",                                                                                            
     "score_explanation": "The call was handled in a reasonable timeframe, but the CSR did take some time to verif 
 flight details."                                                                                                  
   },                                                                                                              
   "Adherence_to_Policies_and_Procedures": {            

In [ ]:
import json
from pathlib import Path

response = json.loads(parsed_evaluation.json())
response["call_ID"] = transcript_dict['call_ID']
response["CSR_ID"] = transcript_dict['CSR_ID']
response["call_date"] = call_date
response["call_time"] = call_time
response = json.dumps(response, indent=2)

pretty_print_json(response)



C:\Users\utente\AppData\Local\Temp\ipykernel_1696\3532956461.py:5: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response = json.loads(parsed_evaluation.json())


                                                                                                                   
 {                                                                                                                 
   "Communication_Skills": {                                                                                       
     "score": "Medium",                                                                                            
     "score_explanation": "The CSR communicated adequately but could have been more concise and used more empathet 
 language when addressing the customer's frustration."                                                             
   },                                                                                                              
   "Problem_Resolution": {                                                                                         
     "score": "High",                                                                                              
     "score_explanation": "The CSR resolved the customer's issue by offering an immediate refund and finding an    
 alternative flight that met the customer's needs."                                                                
   },                                                                                                              
   "Product_Knowledge": {                                                                                          
     "score": "High",                                                                                              
     "score_explanation": "The CSR showed a strong understanding of the company's policies and was able to clearly 
 explain the details of the flight refund process."                                                                
   },                                                                                                              
   "Professionalism": {                                                                                            
     "score": "High",                                                                                              
     "score_explanation": "The CSR maintained a professional demeanor throughout the conversation, addressing the  
 customer's concerns respectfully and patiently."                                                                  
   },                                                                                                              
   "Problem_Escalation": {                                                                                         
     "score": "Medium",                                                                                            
     "score_explanation": "The CSR acknowledged the issue but did not transfer the call to a supervisor until      
 further prompting from the customer."                                                                             
   },                                                                                                              
   "Resolution_Follow_Up": {                                                                                       
     "score": "High",                                                                                              
     "score_explanation": "The CSR mentioned the follow-up procedure and confirmed that an email would be sent to  
 the customer for further details."                                                                                
   },                                                                                                              
   "Efficiency": {                                                                                                 
     "score": "Medium",                                                                                            
     "score_explanation": "The call was handled in a rea

In [166]:
sanitized_call_date = call_date.replace(':', '-').replace(' ', '_')
sanitized_call_time = call_time.replace(':', '-').replace(' ', '_')

In [ ]:
output_directory = "output" 
output_path = Path(output_directory) / f"Call_score_{sanitized_call_date}_{sanitized_call_time}.json"


In [ ]:
try:
    with open(output_path, 'w', encoding='utf-8') as file:
        file.write(response)
    print(f"Transcript score assessment written to local file: {output_path}")
except Exception as e:
    print(f"An error occurred while writing to the file: {e}")

Transcript score assessment written to local file: output\Call_score_2024-02-01_02-16-43.json
